In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Double-pass Acusto-optic modulation

Acusto-Optic Modulators use the acusto-optic effect to diffract and shift the frequency of light using sound waves.\
The mechanism is fairly straightforward. A piezoelectric transducer is attached to a crystal, such as quartz, and the transducer is made to vibrate with a RF signal, thus producing sound waves in the crystal. This in turn modulates the diffraction index of the crystal producing Bragg diffraction of the incoming beam as a result.\
Alternatively we can think of phonon-photon interaction: depending on how many phonons a photon absorbs or emits (stimulated emission) we have and outgoing photon with different momenta and frequency as the incoming one due to momenta and energy conservation.

<img src="Images/AOM.jpg" alt="drawing" width="300"/>

If you use an AOM in double pass configuration it means that the laser beams goes through the AOM twice, thus if you consider the same diffraction order in both direction you get:
- double the frequency shift: for example, with +1 order in both directions and AOM frequency $F$ you get a total frequency shift of $\Delta f = 2\cdot(+1)F = +2F$
- zero net momentum shift: for example, with +1 order in both directions for the first pass you get $+k_F$, but when it is reflected it becomes a $-k_F$ shift, so the second (reflected) pass gives another $+k_F$ shift, thus effectively giving zero net momentum shift

The double-pass configuration is thus very useful to shift a laser frequency in the $\sim 100MHZ$ range without affecting the alignment of the beam itself.

This is very useful to create a pump and probe beams scannable in frequency.

## Electronics

### RF Signal Generator

IMPORTANT: EIT is a coherent phenonema, so phase is critical to see a signal. Therefore, using two simple Voltage Controlled Oscillator (VCOs) in principle would not work, as the pump and probe AOM RF signals would not be phase-locked. So we have a function generator, capable of generating phase locked signals up to 100MHz.

Our aim is to send a sinusoidal signal at a frequency $F$ with a power $P_{RF}$ to the RFA.\
While the frequency is trivial to set, the power is not so trivial. 
Function generators (FG) always have an output resistance of 50Ohms for impedence matching, so we can think of a FG in reality as a voltage source with a 50 OHm resistance in series. FG usually have two output modes:

- 50 Ohm : The FG provides double the voltage set, as it assumes that the load resistor is 50 Ohm, thus giving a 1/2 voltage divider

We understand this thinking that the aim of the FG is to provide the voltage set at its output (ie after the output resistor of the FG itself).\
To avoid confusion we set our FG to "High Z" output. What is the power provided then?\
If we attach the FG to a 50 Ohm load we then have in first approximation the following simple circuit:

<img src="Images/FG_circuit.jpg" alt="drawing" width="300"/>

The power through the load resistor will then be:

$$ P_{load} = \left(\frac{V_{set}}{R_{fg}+R_{load}}\right)^2 R_{load} $$
$$ P_{load}[mW] = 5 \cdot (V_{set}[V])^2 $$

If we now consider a sinusoidal signal of amplitude $V_{amp}$ its $V_{rms}$ (ie square root of average square voltage) will be:

$$ V_{rms} = \frac{V_{amp}}{2\sqrt{2}} $$

The $V_{rms}$ is what we are interested in as it will give us the average power.

Moreover instruments such as amplifiers have specs in dBm, which are simply:

$$ P[mW] = 10^{dBm/10} $$

$$ dBm = 10 \log_{10}P[mW] $$ 

We then ask ourselves which is the $V_{amp}$ we need to set to get a certain power in dBm, for a $50\Omega$ load:

$$ V_{amp}[V] = \frac{2\sqrt{10}}{5}10^{dBm/20} \approx 1.26 \cdot 10^{dBm/20}  $$


### RF Attenuator

The attenuator is a voltage-controlled deviced used to take an input signal and output the same signal but with a certain attenuation.

Our RFA has an attenuation range of approximately $[-20, -5] Db$.

This means that if our input RF signal has $x dBm$ then we can have an output signal in the range $ [x-20,x-5] dBm $ .

### Amplifier
The amplifier takes an input signal and amplifies it into the output signal. \

The important specs are:
- Frequency range: do not operate the amplifier with signals out of the range. Our amplifier has $f_{min} = 0.4MHz,f_{max}=150MHz$
- Gain: expressed in dB, indicates what is the signal amplification. Our amplifier has $G=24dB$
- DC supply voltage: indicates what is the supply voltage the amplifier needs to operate. Our amplifier has $V_{supply} = 24V$
- RF input power (no damage): indicates what is the max power of the input signal in order not to damage the amplifier. Our amplifier has $+20dBm$.

What is the output power in watts given an input of dBm? 

$$ G[dB] = 10 \log_{10}\left(\frac{P_{out}}{P_{in}}\right) $$

So we get:

$$ P_{out}[mW] = 10^{(P_{in}[dBm]+G[dB])/10} $$

$$ P_{in}[dBm] = 10 \log_{10}(P_{out}[mW])-G[dB] $$

### Power supply

We are using a power supply with fixed voltage values. We need to set the 12V option, so that the "+" end will be at +12V and the "-" end will be at -12V

### Acusto-Optic Modulators (AOMs)


The AOM relevant specs are:

- Central drive frequncy: it's the RF where the diffraction efficiency is maximum. Our AOM has $F_c = 80MHz$
- AO Bandwidth: it's the bandwidth where you are supposed to operate the AOM. Our AOM has $DF = 30MHz \Rightarrow F_{range} = [65MHz,95MHz]$ 
- Optical range: range of wavelenghts where the AOM works. Our AOM has 700-1100nm
- Laser beam diameter: maximum beam size for optimal operation (otherwise the beam clips). Our AOM has 1.5mm
- Max Accepted Power: maximum power to drive the AOM. Our AOM has 2.2W

We need then to make sure of two things:
- the driving power does not exceed the Max accepted power. We choose max power of 2W
- the beam size is smaller than the maximum. We choose beam size 1mm or lower.

## Electronics implementation

The purpose is to provide the correct RF signal to the AOM.
For the frequency simply select the one you want in the function generator.

For the power you need to compute what is the power you want in the RF signal, if we consider:
- max 2W out of amplifier (well withing AOM specs), so 9dBm amplifier input (well within amplifier specs)
- RFA from -20 to -5 Db, so RFA input 14dBm

So with our components we need to set $V_amp \approx 6V$ on our RF function generator in High Z mode.

With the RF generator output turned off, we connect the RF generator output to the RFA input.
Take care that for now the RFA remains off.

Now that we have set our function generator we have to connect everything correctly.
ORDER IS IMPORTANT, OTHERWISE YOU RISK BURNING EXPENSIVE COMPONENTS

1. Connect the amplifier input to the RFA output, and the amplifier output to the AOM input
2. Connect the amplifier to the power supply with the banana cables (connect to plus and minus in power supply, not to ground)
3. Set the power supply to 12V and turn it on

Now you can turn on the RF generator output.

Now you can turn on and off the AOM only with the RFA small lever (from off to on).

### "Turning Off"

The order in which you turn off things is also critical:
1. Turn off RF signal through RFA lever
2. Turn off power supply

Only now you can in principle disconnect the cables of the input and output of the amplifier if you need.

## Optics Implementation
An AOM in double-pass configuration is as follows:
1. Initial PBS: needed to separate the incoming beam (p-polarized) from the retroreflected double-passed beam (s-polarized).
2. Two mirrors: used to allineated the incoming beam in the AOM (may be avoided but then allineation is much more messy)
3. AOM: obviously
4. Lense (we have f=100mm): Used to focus the beam on the mirror, it is placed at f-distance from both AOM hole and mirror
5. QWP: used to change the polarization of the double passed beam from p to s (since beam goes through twice a QWP acts as a HWP)
6. Iris: used to select a certain diffraction order, can be substituted with a fixed mechanical block (like black metallical foil) once the setup does not need to be changed any more.
7. Mirror: used to retroreflect the beam

<img src="Images/doublepass_optics.png" width=500 />

### 0 - Placing the first parts

You want to place the PBS, the two mirrors and the AOM.

For placing the PBS:
- The PBS should be "orthogonal" to the breadboard lattice: check that the reflected beam follows the breadboard lattice
- the beam should be centered in the PBS, so that following alignemnts will be unlikely to cause clipping

For placing the mirrors:
- the beam should be centered in the mirror: better polarization and more "room to manouver when aligning
- place the mirror with the screws in neutral position, and do a rough alignment of the mirror when placing it (ie reflected beam follows breadboard lattice)

For placing the AOM:
- AOM is bidirectional, place it in the direction that makes the most sense to the setup space-wise

### 1 - Gross beam alignment

Use the two mirrors before the AOM to center the beam inside the AOM hole. You can check that the alignment is good enough as the efficiency (power before vs after AOM hole) is above 95%.
If you cannot get high efficiency it could be that you are either
- clipping your beam on the AOM hole: center the beam better
- beam size is too big: check with camera the beam size and compare to AOM specs, if needed use lenses to change beam size

### 2 - Place the lense

Place the lense at a focal length distance from the AOM hole. Try to center the beam on the lense as best as possible to have a cleaner beam profile with less aberrations.

### 3 - Finding the Bragg-angle

Incline the AOM with the mounts screws until you see diffraction. Take into consideration that it could take a while, but if you prepared all the electronics correctly you will see diffraction.\
It could be that during this process the AOM hole moves in such a way that the beam gets clipped, if this happens use the two mirrors before the AOM to correct.

### 3.5 - Understanding which order is which

To understand what is the +1 and -1 order think like this: the RF signal comes from up, so the phonons are "traveling" down, so if the photon absorbs one phonon (+1) it gets "kicked down" while if it emits a phonon (-1) it gets kicked up.

Also, once you see doppler-broadened spectroscopy in both cells (EIT and SAS) you can simply see the frequency shift of the AOM (ie negative shift (-1) you see EIT signal shifted to the right )

### 4 - 0th order alignment

Use the two mirrors before the AOM to optimize the 0th order efficiency (total power before vs after AOM hole). If you do not get high efficiency it could be for one of the reasons mentioned before.

### 5 - Iris Placement

Place the iris to block the 0th order and let through either the +1 or -1.
To check that you placed the iris correctly measure the power before and after the iris with the RF off, even with mW of power before the iris you should still get only tens of uW after the iris. Then open the iris as much as you can while this remains true in order to maximize the power of the +1/-1 order that gets through the iris.

### 6 - Optimizing the 1st order

Use the AOM mount screws to optimize the power after the iris with RF on.
Be mindful of the beam position: while doing this process the position will change, and you will then need to move the powermeter so that the beam remains in the center of the powermeter.
Once you optimized the AOM inclination optimize the RF power through the RFA, you should expect to almost match the declared efficiency (power before AOM vs after iris) if you have done everything correctly. 
Eg for our AOMs you should get above 85% efficiency at 80MHz

### 7 - Placing the QWP and the mirror

Place the QWP after the iris:
- make sure the beam is centered so that there is no clipping, you can make sure of this by measuring the power before vs after the QWP

Place the mirror:
- make sure the beam is centered in the mirror with screws in neutral position
- try to place the mirror as orthogonal to the beam as possible

### 8 - Aligning the double pass

Take a piece of white paper and make a small hole.
Turn on the power of the beam so that it is easily visible on the white piece of paper
Now place the piece of paper before the mirror so that the beam goes through the hole, you will see a retroreflected beam.
Use the mirror mount screws to overlap incoming and retroreflected beam (ie retroreflected beam should go through the hole)
Now move the piece of paper before the QWP and repeat, continue to move the piece of paper and repeat until you get before the initial PBS
Now you should see the retroreflected beam out of the initial PBS. Rotate the QWP until the power of the retroreflected beam out of the PBS is maximum.

### 9 - Selecting the double-passed component

The retroreflected beam also gets diffracted by the AOM, so after the PBS you will see more than one beam. To understand which one is the one you want place yourself far from the PBS and look at the beams with the infrared card, now change the RF frequency: the beam that does not change height is the one you want.
Block the other beams with an iris or in some other mechanical way
Theoretically if you now check the efficiency before AOM vs double-passed beam you should get the square of the single pass efficiency.

## Improvements

### PID Stabilization

When modulating the AOM frequency the signal gets very noisy, as the efficiency of the AOM is not frequency-independent.

<img src="Images/AOM_noise.jpeg" alt="drawing" width=300/>

Implementing a PID that stabilizes the probe power before the EIT cell could help in cleaning up the signal. Without this, unless the signal is extremely clear it may not possible to see it directly on the oscilloscope. The PID should have control on the analog input of the RFA.

However the power of the beam that that gets sampled for stabilization needs to have the same (or more) power than the beam that goes to the cell, otherwise the residual power noise gets "amplified in a way"

<img src="Images/PID_noise.jpg" width=300/>

Here we have the sampled beam power (blue) much stronger than the cell power (yellow), so even if the PID does a good job of stabilizing the power the signal we are interested in is still very noisy.

An alternative would be to record both the signal after the cell and the signal before the cell, in order to be able to post-process the data correcting for the power modulation of the probe as we are sweeping it in frequency